In [1]:
#will go into .gitignore
import numpy as np
import json 
import logging

In [2]:

logging.basicConfig(filename='log',level=logging.DEBUG)
#numpy writer::
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
def loss_cal(prob,i,j,state): #i is position of last query, j is last state at which we need loss
    
    n=prob.shape[1]
    loss = np.zeros((n,1));
    # print(loss.shape)
    for x in range(j-i):
        temp = np.reshape(np.min(np.linalg.matrix_power(prob,x+1),axis=1),(2,1))
        # print(temp.shape)
        loss+= temp
    return np.matmul(state,loss)

def query_loss(i,j,res,prob):
        #this is a quey to return loss from i-j inclusive if a query is made given outcome
        state_curr=np.array([[1-res],[res]])
        loss=0
        for idx in range(i+1,j+1):
            state_curr = np.matmul(prob,state_curr)
            loss+=np.min(state_curr)
        # print("I: ",i,"J: ",j,"LOSS: ",loss)
        return loss

def write_file(fp,json_obj):
    json_=json.dumps(json_obj,indent=4,cls=NumpyEncoder)
    with open(fp,'a') as f:
        f.write(json_)

def write_log(log,type='info'):
    if(type=='info'):
        logging.info(log)
    elif(type=='debug'):
        logging.debug(log)
    elif(type=='warning'):
        logging.warning(log)
    elif(type=='error'):
        logging.error(log)
    elif(type=='critical'):
        logging.critical(log)


def read_ndjson(fp):
    json_str=""
    arr=[]
    with open(fp) as f:
        for line in f:
            if(line[0]=='}' and line[1]=='{'):
                arr.append(json.loads(json_str))
                json_str="{"
            else:
                json_str+=line
    return arr

In [3]:
def OPTPolicy(prob,T,cost):
    n = prob.shape[0]
    
    dp = np.zeros((T+1,n)) # dp_ik is the position to query next if previous query was at i and result was state k   
    loss = np.zeros((T+1,n)) # Expected loss till the end of the horizon
    for i in range(n):
        dp[T][i]=np.inf
        loss[T][i]=0

    for i in range(T-1,-1,-1):
        for k in range(n):
            state = np.zeros((1,n))
            state[0,k]=1
            curr=loss_cal(prob,i,T,state)
            dp[i,k]=np.inf
            for x in range(i+1,T+1):
                new_curr = loss_cal(prob,i,x-1,state) + cost
                
                state = np.matmul(state,np.linalg.matrix_power(prob,x-i))
                for s in range(n):
                    new_curr += loss[x,s]*state[0,s]

                if new_curr < curr:
                    curr=new_curr
                    dp[i,k]=x
                    
            loss[i,k]=curr
            
    return loss[0,0]
        
def DP(mdp_history,prob,cost,k_max=100):
    horizon=np.size(mdp_history)
    arr=np.zeros((horizon,k_max+1,horizon))+np.inf# DP array i,j,k represents min cost of [0-i] with j vals and last probe at k::

    for i in range(horizon):
        print(f"{i}/{horizon}")
        for k in range(i+1):
            for j in range(min(k+2,k_max+1)):
                if(j==0):
                    arr[i,j,k]=query_loss(0,i,mdp_history[0],prob)
                elif(k==i):
                    if(i==0):
                        arr[i,j,k]=min(arr[i,j,k],0+cost)
                    else:
                        arr[i,j,k]=min(arr[i,j,k],np.min(arr[i-1,j-1,max(j-2,0):i])+cost)
                elif(k<i):
                    arr[i,j,k]=min(arr[i,j,k],arr[k,j,k]+query_loss(k,i,mdp_history[k],prob))
            #k_max=int(np.min(arr[arr>=0])//cost)#updating k_max at the end of each iteration::# can be improvised
    best_loss=(np.min(arr[horizon-1,:,:]))
    #=print(best_loss)
    return best_loss

In [4]:
class Simulator:
    def __init__(self, prob, state, horizon, cost):
        self.prob = prob
        self.state = state
        self.horizon = horizon
        self.cost = cost
        self.num = prob.shape[0]  #number of states
        self.history=[state]
    def next(self):
        tr=np.random.rand()
        
        tot=0.0
        
        for i in range(self.num):
            if tr<=tot+self.prob[self.state][i]:
                self.history.append(i)
                return i
            else:
                tot+=self.prob[self.state][i]
        #write_log(f"tr:{tr},tot:{tot}")

    def simulate(self):
        
            
        for step in range(self.horizon):
            self.state = self.next()
            
        
        return np.array(self.history)


In [5]:
def test_algo(algo_arr,test_cases=None):
    #tests against all the algoriths provided:: in the algo arr against THE conjectured algorithm::
    if(test_cases is None):
        test_cases=[]
        for i in range(50):
            dict_test_case={}
            p1=np.random.rand()
            p2=np.random.rand()
            dict_test_case['prob']=np.array([[p1,1-p1],[p2,1-p2]])
            dict_test_case['T']=10*np.random.randint(1,20)
            dict_test_case['cost']=3*np.random.rand()
            test_cases.append(dict_test_case)
     
    for test_case in test_cases:
            #making a history obj that will be stored in a file:: for graphs and evaluation::
            loss=0
            for i in range(100):
                 sim = Simulator(test_case['prob'],0,test_case['T'],test_case['cost'])
                 history=sim.simulate()

                 loss_p_case=DP(history,test_case['prob'],test_case['cost'])
                 loss+=loss_p_case
            print(f"expected DP loss:{loss/100}\n loss from algo: {OPTPolicy(test_case['prob'],test_case['T'],test_case['cost'])}")

In [2]:
#test_algo([])   
#formulating problem as constraint minimization problem::
def get_l(state,alpha,prob,cost):
    loss=0
    mtx=np.array([1-state, state])
    for i in range(alpha-1):
        mtx=np.matmul(mtx,prob)
        loss+= np.min(mtx)
    mtx=np.matmul(mtx,prob)
    return loss+cost,mtx
Mx_val=10
arr=np.zeros((Mx_val,Mx_val))
prob=np.array([[0.8,0.2],[0.1,0.9]]) 
cost=0.4
for i in range(1,Mx_val+1):
    for j in range(1,Mx_val+1):
        alpha_l,P_0=get_l(0,i,prob,cost)
        beta_l,P_1=get_l(1,j,prob,cost)
        v=(alpha_l*P_1[0]+P_0[1]*beta_l)/(i*P_1[0]+j*P_0[1])
        arr[i-1,j-1]=v
        #print(v,end=" ")
    #print("")
res=np.unravel_index(np.argmin(arr),arr.shape)
print(arr)
alpha=res[0]+1
beta=res[1]+1
print(f"alpha:{alpha} \nbeta:{beta}")
    

[[0.4        0.29473684 0.27057387 0.26499573 0.26570214 0.26876664
  0.27264382 0.27666471 0.28053357 0.28412741]
 [0.36296296 0.26666667 0.24636488 0.24335155 0.24605832 0.25079106
  0.25610188 0.26137279 0.26634027 0.27090465]
 [0.36476965 0.27330447 0.25333333 0.24980453 0.25182542 0.2558887
  0.26060084 0.26535438 0.26988101 0.27407151]
 [0.37551291 0.28795181 0.2676367  0.263      0.26379985 0.26671004
  0.27038847 0.27423501 0.2779728  0.28147897]
 [0.38780603 0.30391785 0.28324453 0.27750024 0.27706667 0.27879734
  0.28140524 0.28430152 0.28720397 0.28997838]
 [0.39299633 0.31321631 0.29281548 0.28656237 0.28542832 0.28644444
  0.28838467 0.2906792  0.29304822 0.29535318]
 [0.39455283 0.31892518 0.29909328 0.29264296 0.29109482 0.29164961
  0.29314286 0.29502712 0.29702876 0.29900862]
 [0.39417338 0.32256075 0.30343204 0.29695549 0.29515852 0.29540105
  0.29657846 0.29816667 0.29990015 0.30164115]
 [0.39276172 0.3249425  0.30656663 0.30015963 0.29821383 0.29823692
  0.29918116 

In [6]:
#checking the assumption @1
#--> optimal alpha and beta are independent of each other::
cond=True
for i in range(arr.shape[0]):
    idx=np.argmin(arr[i])
    cond=cond and (idx+1==beta)
    print(idx+1,beta)
    print("-----------")
print("<><><><><><><><><>")
for i in range(arr.shape[1]):
    idx=np.argmin(arr[:,i])
    cond=cond and (idx+1==alpha)
    print(idx+1,alpha)
    print("----------")
print(cond)
#assumption turns out to be false::

4 4
-----------
4 4
-----------
4 4
-----------
4 4
-----------
5 4
-----------
5 4
-----------
5 4
-----------
5 4
-----------
5 4
-----------
6 4
-----------
<><><><><><><><><>
2 2
----------
2 2
----------
2 2
----------
2 2
----------
2 2
----------
2 2
----------
2 2
----------
2 2
----------
2 2
----------
2 2
----------
False
